## 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

> Trong notebook này, chúng ta sẽ cùng implementing model từ paper [ Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/pdf/1406.1078.pdf). Model này sẽ cải thiện độ chính xác trên tập test một cách đáng kể trong khi chỉ sử dụng một RNN layer ở encoder và decoder.

### Introduction

> Cùng nhìn lại kiến trúc chung của encoder-decoder model.

![figure1](./images/2.seq2seq_model.PNG)

> Các tokens đầu vào đi qua embedding layer (yellow), sau đó đi vào encoder (green) để tạo ra *context vector* (red). Sau đó, *context vector* được đưa vào decoder (blue), sinh ra các vector và được đưa vào linear layer (purple) để sinh ra target sequence.

> Ở notebook trước, chúng ta sử dụng multi-layered LSTM để xây dựng encoder và decoer.

![figure2](./images/2.lstm_seq2seq.PNG)

> Nhược điểm của model trong notebook trước là decoder cố gắng nhồi nhét quá nhiều thông tin vào trong hidden states. Trong quá trình decoding, hidden state cần phải chứa TOÀN BỘ thông tin về source sequence, cũng như toàn bộ các tokens đã được decoded. Bằng cách giảm NHẸ lượng thông tin này, ta có thể tạo một model tốt hơn!

> Trong notebook này, ta sẽ sử dụng GRU (Gated Recurrent Unit) thay cho LSTM (Long Short-Term Memory). Tại sao? Lý do chính là bài báo mà chúng ta khảo sát sẽ sử dụng GRUs, và notebook trước chúng ta đã dùng LSTMs rồi :v. Để hiểu về sự khác nhau giữa GRUs, LSTMs và standard RNNs, nhấn vào [đây](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Vậy GRUs hay LSTMs tốt hơn? [Nghiên cứu này]() chỉ ra rằng GRUs và LSTMs cho kết quả xấp xỉ nhau, và cả 2 đều tốt hơn standard RNNs.

### Preparing Data

> Chúng ta sẽ sử dụng torchtext để thực hiện các bước pre-processing và dùng spacy để tokenize data.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

> Chúng ta sẽ set random seeds để thu được kết quả đồng nhất ở các lần chạy.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Tiêp theo, ta tạo tokenizers. Tokenizer được sử dụng để biến một string chứa các sentence thành một list các tokens riêng biệt tạo nên string đó. ví dụ: "good morning!" sau khi được tokenized trở thành ["good", "morning", "!"].

> spaCy có các models để tokenize cho các loại ngôn ngữ khác nhau. Với tiếng Đức ta có "de_core_news_sm", tiếng Anh là "en_core_web_sm".

> **NOTE:** Các models tokenize cần được tải về. <br>
>> * python -m spacy download en_core_web_sm <br>
>> * python -m spacy download de_core_news_sm

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

> Ta cần tạo hàm để tokenize các sentences. Hàm này sẽ có input là string, output là list of tokens.

> Lưu ý: Hàm tokenize tiếng Đức sẽ đảo thứ tự list sau khi được tokenize do việc đâỏ ngược thứ tự các tokens giúp quá trình optimization dễ dàng hơn.

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (as tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (as tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

> `Field`s trong torchtext sẽ xác định cách mà data được xử lý. Nhấn vào [đây]() để biết thêm chi tiết về `Field`.

> Ta cần set các đối số `tokenize` đúng với hàm tokenize của nó. Tiếng Đức tương ứng với biến `SRC`, tiếng Anh là `TRG`. Các trường cũng chèn thêm tokens 'start of sequence' và 'end of sequence' thông qua `init_token` và `eos_token`, đồng thời nó sẽ conert tất cả các từ sang lowercase.

In [5]:
SRC = Field(tokenize = tokenize_de,
            init_token = '',
            eos_token = '',
            lower = True)

TRG = Field(tokenize = tokenize_en,
            init_token = '',
            eos_token = '',
            lower = True)

> Tiếp theo, chúng ta sẽ tải và load train, test, validation dataset.

> Dataset ở đây là [Multi30k dataset](https://github.com/multi30k/dataset). Dataset chứa ~30000 câu tiếng Anh, Đức, Pháp và mỗi câu có khoảng 12 từ.

> `exts` xác định ngôn ngữ nào được sử dụng cho source và target (source đứng trước) còn `fields` xác định field nào được sử dụng cho source và target.

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

> Check data.

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


> Hiển thị một câu.

In [8]:
vars(train_data.examples[0])

{'src': ['.',
  'büsche',
  'vieler',
  'nähe',
  'der',
  'in',
  'freien',
  'im',
  'sind',
  'männer',
  'weiße',
  'junge',
  'zwei'],
 'trg': ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.']}

> Tiếp theo, chúng ta sẽ xây dựng *vocabulary* cho source và target languages. Vocabulary dùng để liên kết mỗi unique token với chỉ số của nó. Vocabulary của source và target languages là khác nhau.

> Sử dụng đối số `min_freq = 2`, ta chỉ cho phép các tokens xuất hiện tối thiểu 2 lần được đưa vào vocabulary. Các tokens xuất hiện ít hơn 2 lần sẽ được coi là \<unk> hay unknow token.

> Một điều cần lưu ý nữa là ta chỉ xây dựng vocabulary cho training set, không xây cho validation/test set. Việc này giúp ngăn chặn vấn đề "information leakage" (information leakage khiến việc đánh giá không được khách quan).

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [10]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7854
Unique tokens in target (en) vocabulary: 5892


> Bước cuối cùng trong tiền xử lý data là tạo iterators. Đối tượng iterators này sẽ được lặp trong từng batch và có `src` attribute (là một Pytorch tensors chứa một batch các source sentences đã được số hóa - Numericalized), có `trg` attribute (là một Pytorch tensors chứa một batch các target sentences đã được số hóa - Numericalized). Numericalized là một cách nói đơn giản của việc chuyển các chuổi tokens có thể đọc được thanh chuối các indexes tương ứng, sử dụng vocabulary.

> Khi ta train, trong 1 batch luôn yêu cầu các sentence phải có cùng độ dài, do đó ta cần padding các câu ngắn. May mắn thay, torchtext có các công cụ để xử lý vấn đề này.

Ta sử dụng `BucketIterator` thay cho standard `Iterator` vì nó tạo các bathes theo cách tối ưu nhất - tối thiểu hóa số lượng padding token ở cả source và target sentences.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

### Building the Seq2Seq Model

##### Encoder

> Encoder tương tự như notebook trước. Có 2 điểm khác biệt so với notebook trước là: LSTM được thay thế bởi single-layer GRU và chỉ sử dụng 1 layer thay vì 2 layers. Bên cạnh đó, ta sẽ không truyền tham số `dropout` vào RNN do chỉ sử dụng 1 lớp. Nếu cố gắng truyền `dropout` khi sử dụng 1 layer, Pytorch sẽ warning.

> Một điểm cần lưu ý khác là GRU chỉ yêu cầu hidden state, đồng thời nó cũng trả về hidden state, cell state không tồn tại trong GRU.

> Dưới đây là biểu thức của GRU, LSTM và standard RNN.

>> * $\bold{h_t}$ = $\bold{GRU(e(x_t), h_{t-1})}$

>> * $\bold{(h_t, c_t)}$ = $\bold{LSTM(e(x_t), (h_{t-1}, c_{t-1}))}$

>> * $\bold{h_t}$ = $\bold{RNN(e(x_t), h_{t-1})}$

> Từ biểu thức trên, ta có thể thấy RNN và GRU khá giống nhau. Tuy nhiên, bên trong GRU có một cơ chế gọi là *gating machanisms* điều khiển lượng thông tin vào và ra hidden state (tương tự như LSTM). Nhấn vào [đây](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) để đọc thêm về GRU và *gating machanisms*.

> Phần còn lại của encoder tương tự so với tutorial trước, nó nhận đầu vào là một sequence, $\bold{X = \{x_1, x_2, ..., x_T\}}$ được truyền qua embedding layer, được tính hidden states $\bold{H = \{h_1, h_2, ..., h_T\}}$ và trả về context vector (final hidden state), z = $h_T$.

>> $\bold{h_t = EncoderGRU(e(x_t), h_{t-1})}$

> Encoder được tạo bởi GRU tương tự như encoder của các seq2seq model thông thường.

![gru](./images/2.seq2seq_gru.PNG)

In [12]:
class Encoder(nn.Module):
    def __init__(self,
                input_dim,
                emb_dim,
                hid_dim,
                n_layers = 1,
                dropout=0.5):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        if n_layers > 1:
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout)
        else:
            self.rnn = nn.GRU(emb_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]

        # outputs are always from the top hidden layer

        return hidden

##### Decoder

> Decoder trong notebook này sẽ có nhiều thay đổi so với decoder model ở notebook trước, và chúng ta sẽ giảm việc nén thông tin.

> GRU trong decoder sẽ nhận đầu vào là embedded target token, $\bold{d(y_T)}$, previous hidden state $\bold{s_{t-1}}$ và context vector z.

>> $\bold{s_t = DecoderGRU(d(y_t), s_{t-1}, z)}$

> Lưu ý là context vector z không có chỉ số $\bold{t}$, có nghĩa là ta sẽ sử dụng CÙNG một context vector được trả về từ encoder trong suốt quá trình decoder.

> Trước đây, chúng ta dự đoán token tiếp theo, $\bold{\hat{y}_{t+1}}$ bằng linear layer **f**, và chỉ sử dụng top-layer decoder hidden state ở thời điểm đó, $s_t$ như sau: <br>
>> $\hat{y}_{t+1}$ = f($s^L_t$) <br>

> Giờ đây, ta còn truyền thêm embedding của token hiện tại, $d(y_t)$ context vector z vào linear layer để dự đoán. Biểu thức như sau: <br>
>> $\hat{y}_{t+1}$ = f($d(y_t)$, $s^L_t$, z)

> Kiến trúc decoder của chúng ta sẽ trông như thế này:

 > ![decoder](./images/2.seq2seq_decoder.PNG)

> Lưu ý, initial hidden state, $s_0$ vẫn là context vector z. Cho nên khi sinh token đầu tiên, thực chất chúng ta sẽ đưa 2 vector giống hệt nhau vào GRU.

> Câu hỏi đưa ra là: làm cách nào mà 2 thay đổi này giảm việc nén thông tin lại? Well, theo lý thuyết thì decoder hidden states $s_t$ không cần chứa thông tin về source sequence vì source sequence luôn có sẵn. Do đó, nó chỉ cần lưu trữ các thông tin về token nào đã được sinh ra tính đến thời điểm hiện tại. Việc thêm $\bold{y_t}$ và linear layer cũng có nghĩa là layer này có thể trực tiếp quan sát token đó là gì mà không cần thêm thông tin từ hidden state.

> Tuy nhiên thì lý thuyết vẫn là lý thuyết, ta không có cách nào xác định được model có thực sự sử dụng thông tin được cung cấp hay không? Tuy nhiên, về mặt trực quan, ta có thể thấy rằng việc điều chỉnh model này là một good ideal!

> Khi implementation GRU, ta truyền $d(y_t)$ và z vào bằng cách ghép nối chúng lại với nhau, cho nên input dimensions của GRU sẽ là `emb_dim + hid_dim` (do context vector sẽ có size là `hid_dim`). Linear layer sẽ nhận $d(y_t)$, $s_t$ và z là đầu vào, do đó ta cần concatenating chúng lại và ta có input dimensions sẽ là `emb_dim + hid_dim*2`. Và, đương nhiên là ta không cần truyền giá trị dropout vào GRu vì ở đây, ta chỉ sử dụng duy nhất 1 layer.

> Giờ đây, hàm `forward` nhận thêm 2 đối số là `context` và `hidden`. Bên trong `forward`, ta ghép nối $y_t$ và z thành `emb_con` trước khi đưa vào GRU, sau đó ta concatenate d($y_t$), $s_t$ và z cùng nhau tạo thành `output` trước khi đưa nó vào linear layer để đưa ra dự đoán token, $\hat{y}_{t+1}$.

In [13]:
class Decoder(nn.Module):
    def __init__(self,
                output_dim,
                emb_dim,
                hid_dim,
                n_layers = 1,
                dropout=0.5
    ) -> None:
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        if n_layers > 1:
            self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout)
        else:
            self.rnn = nn.GRU(emb_dim, hid_dim)

        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, context):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # context = [n layers * n directions, batch size, hid dim]

        # n layers and n directions in the decoder will both always be 1, therefore:
        # hidden = [1, batch size, hid dim]
        # context = [1, batch size, hid dim]

        input = input.unsqueeze(0)

        # input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        # embedded = [1, batch size, emb dim]

        output, hidden = self.rnn(embedded, hidden)

        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # seq len, n layers and n directions will always be 1 in the decoder, therefore:
        # output = [1, batch size, hid dim]
        # hidden = [1, batch size, hid dim]

        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)

        # output = [batch size, emb dim + hid dim * 2]

        prediction = self.fc_out(output)

        # prediction = [batch size, output dim]

        return prediction, hidden

##### Seq2Seq Model

> Ghép encoder và decoder vào với nhau, ta có:

> ![model](./images/2.seq2seq_together.PNG)

> Khi implementation, ta cần đảm bảo hidden dimension ở encoder và decoder phải bằng nhau.

> Tóm tắt nhanh các bước trong model: <br>
>> * `outputs` tensor được tạo để lưu trữ tất cả các predictions, $\bold{\hat{Y}}$.
>> * Source sequence $\bold{X}$ được đưa vào encoder, từ đó ta thu được `context` vector.
>> * Giá trị khởi tạo của decoder hidden state được gán bằng `context` vector, $s_0$ = z = $h_T$.
>> * Ta sẽ sử dụng batch of \<sos> tokens làm first `input`, $y_1$.
>> * Sau đó, ta thực hiện decode với vòng lặp:
>>>> * Nhét input token $y_t$, previous hidden state $s_{t-1}$ và context vector z vào decoder.
>>>> * Lưu trữ giá trị dự đoán $\hat{y}_{t+1}$ và new hidden state $s_t$.
>>>> * Tiếp theo, ta sẽ kiểm tra điều kiện thực hiện teacher force, và gán next input với giá trị phù hợp (ground của next token trong target sequence hoặc token có giá trị dự đoán cao nhất).

In [14]:
class seq2seq(nn.Module):
    def __init__(self, encoder, decoder, device) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        # src = [src len, batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden = self.encoder(src)
        context = hidden
        # first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):
            # insert input token embedding, previous hidden state and the context state
            # receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest predicted token from our predictions
            top1 = output.argmax(1)

            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

### Training the Seq2Seq Model

> Phần còn lại của notebook này khá giống lần trước.

> Ta khởi tạo encoder, decoder, seq2seq model và đồng thời đưa chúng vào GPU nếu có. Như đã nói từ trước, embedding dimensions và tỷ lệ dropout giữa encoder và decoder có thể khác nhau nhưng hidden dimensions phải giống nhau.

In [15]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = seq2seq(enc, dec, device).to(device)

> Ta khởi tạo các parameters theo phân phối chuẩn $\bold{N(0, 0.01)}$.

In [16]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

seq2seq(
  (encoder): Encoder(
    (embedding): Embedding(7854, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5892, 256)
    (rnn): GRU(256, 512)
    (fc_out): Linear(in_features=1280, out_features=5892, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

> Đếm số lượng parameters:

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,432,068 trainable parameters


> Ta có thể thấy số lượng parameters nhiều hơn model trước mặc dù chỉ sử dụng một layer RNN. Lý do là ta đã tăng kích thước đầu vào của GRU và linear layer nên số lượng parameters tăng là điều đương nhiên. Tuy nhiên số lượng parameters này chỉ làm tăng thời gian train khoảng 3s/epoch.

> Khởi tao optimizer.

In [18]:
optimizer = optim.Adam(model.parameters())

> Khởi tạo loss function, lưu ý là ta cần ignore padding tokens.

In [19]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [20]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [22]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

> We train model.

In [23]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 22m 45s
	Train Loss: 4.952 | Train PPL: 141.404
	 Val. Loss: 5.012 |  Val. PPL: 150.155
Epoch: 02 | Time: 18m 32s
	Train Loss: 4.267 | Train PPL:  71.316
	 Val. Loss: 5.144 |  Val. PPL: 171.344
Epoch: 03 | Time: 18m 13s
	Train Loss: 3.900 | Train PPL:  49.412
	 Val. Loss: 4.558 |  Val. PPL:  95.345
Epoch: 04 | Time: 18m 34s
	Train Loss: 3.532 | Train PPL:  34.183
	 Val. Loss: 4.242 |  Val. PPL:  69.550
Epoch: 05 | Time: 18m 4s
	Train Loss: 3.203 | Train PPL:  24.596
	 Val. Loss: 3.968 |  Val. PPL:  52.898
Epoch: 06 | Time: 18m 25s
	Train Loss: 2.937 | Train PPL:  18.856
	 Val. Loss: 3.909 |  Val. PPL:  49.868
Epoch: 07 | Time: 18m 15s
	Train Loss: 2.718 | Train PPL:  15.146
	 Val. Loss: 3.742 |  Val. PPL:  42.173
Epoch: 08 | Time: 18m 12s
	Train Loss: 2.494 | Train PPL:  12.111
	 Val. Loss: 3.733 |  Val. PPL:  41.811
Epoch: 09 | Time: 18m 24s
	Train Loss: 2.293 | Train PPL:   9.901
	 Val. Loss: 3.710 |  Val. PPL:  40.834
Epoch: 10 | Time: 17m 25s
	Train Loss: 2.147 | 

> Test model.

In [24]:
model.load_state_dict(torch.load('./models/tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.702 | Test PPL:  40.545 |


> Với giá trị test loss, ta có thể tạm kết luận model này tốt hơn model trước. Đây là một dấu hiệu tốt khi đánh giá một kiến trúc model. Giảm việc nén thông tin có vẻ là một phương pháp tốt, và trong notebook tới, ta sẽ mở rộng nó với *attention mechanism*.

## END